# Question 1

In [1]:
import tensorflow as tf
import os

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
batch_size   = 64
image_height = 128
image_width = 128

train_ratio = 0.8
validation_ratio = 0.1
# evaluation_ratio = 0.1

all_dataset = tf.data.Dataset.list_files("4/*.png") # Get all png files' names

def process_file(png_path):
    info_file_path = tf.strings.join([ # Get each info(txt) file's name
        tf.strings.substr(png_path, 0, tf.strings.length(png_path) - len(".png")), ".txt"
    ])
    infos = tf.strings.split(tf.io.read_file(info_file_path), '\t') # Split class, x, y, width, height
    infos = tf.map_fn(lambda x:tf.strings.to_number(x, tf.float32), infos, tf.float32) # Convert infos to float type
    
    image = tf.io.decode_png(tf.io.read_file(png_path), channels = 1) # Read png file
    
    classification = tf.one_hot(tf.cast(infos[0], tf.int32), depth = 10) # Convert class to int32 type and one-hot tensor
    
#    return image, (classification, infos[1], infos[2], infos[3], infos[4])
    return image, classification

# infos[1]: x axis
# infos[2]: y axis
# infos[3]: height?
# infos[4]: width?

all_dataset = all_dataset.map(process_file)

train_size  = int(train_ratio * len(all_dataset))
valid_size  = int(validation_ratio * len(all_dataset))

train_dataset = all_dataset.take(train_size)
valid_dataset = all_dataset.skip(train_size).take(valid_size)
eval_dataset  = all_dataset.skip(train_size).skip(valid_size) # evaluation_ratio = 0.1
train_dataset

Instructions for updating:
Use fn_output_signature instead


<TakeDataset shapes: ((None, None, 1), (10,)), types: (tf.uint8, tf.float32)>

In [3]:
input_dim  = 128
output_dim = 10

lstm_model = keras.Sequential()
lstm_model.add(layers.LSTM(64, input_shape = (None, input_dim)))
lstm_model.add(layers.Dense(output_dim, activation = 'softmax', name = "Classification"))
lstm_model.summary()

ValueError: Input 0 of layer lstm is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, None, 128, 128)

In [ ]:
lstm_model.compile(
    loss      = keras.losses.CategoricalCrossentropy(),
    optimizer = "sgd",
    metrics   = ["accuracy"],
)

lstm_model.fit(train_dataset, epochs = 10, validation_data = valid_dataset, batch_size = batch_size)

In [ ]:
results = model.evaluate(eval_dataset, batch_size = 128)
print("test loss, test acc:", results)